In [1]:
# Installing dependencies
#!pip install praw
#!pip install psaw
#!pip install yfinance

In [2]:
import praw
from psaw import PushshiftAPI
import json
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import traceback
import tensorflow as tf
import numpy as np

# Suppress warning messages
import warnings
#warnings.filterwarnings('ignore')

In [3]:
# Load client_id, secret_id, and user_agent
with open('info.json') as f:
     info = json.load(f)
        
info = dict(info)

In [4]:
# Initialize Reddit and PushshiftAPI instances
reddit = praw.Reddit(client_id=info["client_id"], user_agent=info["user_agent"], client_secret=info["client_secret"])
api = PushshiftAPI(reddit)

In [5]:
# Store results of a search in a DataFrame
"""
subm_dicts = [{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in api.search_submissions(subreddit='stocks', q="TWTR", filter=['url','author', 'title', 'subreddit'], limit=100)]
df = pd.DataFrame(subm_dicts)
df
"""
_ = None

In [6]:
# Get historical stock data for a ticker
twtr = yf.download('TWTR', progress=True)
twtr

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701600
2013-11-08,45.930000,46.939999,40.689999,41.650002,41.650002,27925300
2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113900
2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316700
2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688300
...,...,...,...,...,...,...
2022-04-04,47.869999,51.369999,46.860001,49.970001,49.970001,268465400
2022-04-05,53.849998,54.570000,50.560001,50.980000,50.980000,217520100
2022-04-06,50.040001,52.869999,49.299999,50.770000,50.770000,159034700


## Processing Tickers

This initial thought process is not great. I decided that looking for specific words wwould not be a good idea, as it takes much of the context out of the comment. (Something that briefly mentions TSLA, but is actually talking about how great MSFT is would be useless in predicting TSLA stock)

In [7]:
# At close, calculate the real and percent change since last close
def get_diff(ticker_data):
    df = ticker_data.copy()
    real = []
    percent = []
    for index, row in df.reset_index().iterrows():
        if(index == 0):
            real.append(0)
            percent.append(0)
        else:
            real.append(row["Close"]-df.iloc[index-1]["Close"])
            percent.append(real[-1]/df.iloc[index-1]["Close"])
    return real, percent

# Get the reddit posts that mention a certain ticker n days before a large change in stock price
def get_pre_change_posts(ticker, ticker_gain, days=1, limit=1000, subreddit="stocks,stockmarket,stocksandtrading,daytrading,investing,stocks_picks,stockstobuytoday"):
    df = None
    for index, row in ticker_gain.iterrows():
        start_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6, days=-days)
        end_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6)
        
        # TODO: Check whether comments would be better than submissions
        
        submissions = api.search_comments(after=start_date, before=end_date, q=ticker, subreddit=subreddit, filter=['url','author', 'title', 'subreddit'], limit=limit)
        if(df is None):
            df = pd.DataFrame([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions])
        else:
            df = df.append([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions], ignore_index=True)
    return df

def word_counts(df, column="body", min_letters=3):
    counts = {}
    for i in list(df[column]):
        for j in i.split(" "):
            j = ''.join(k for k in j if k.isalnum())
            # Exclude words that are likely tickers
            if(j == j.upper() and len(j) > 1 and len(j) <= 5):
                pass
            elif(len(j) < 3):
                pass
            elif(j not in counts.keys()):
                counts[j.lower()] = 1
            else:
                counts.update({j.lower():counts.get(j.lower())+1})
    return counts

def remove_shared_keys(dict_a, dict_b, cutoff=2):
    a = dict_a.copy()
    b = dict_b.copy()
    
    rm_a = []
    rm_b = []
    for i in a:
        if(i in b):
            if(b.get(i) > 2*a.get(i)):
                rm_a.append(i)
            elif(b.get(i) < 2*a.get(i)):
                rm_b.append(i)
            else:
                rm_a.append(i)
                rm_b.append(i)
    for i in rm_a:
        a.pop(i)
    for i in rm_b:
        b.pop(i)
    return a, b

def remove_infrequent_words(dict_a, min_count=2):
    d = dict_a.copy()
    
    to_remove = []
    for i, x in d.items():
        if(x < min_count):
            to_remove.append(i)
            
    for i in to_remove:
        d.pop(i)
        
    return d

# Generate information for a given ticker
def process_ticker(ticker, gain_cutoff=0.05, loss_cutoff=0.05, limit=100, days=1):
    try:
        ticker_data = yf.download(ticker, progress=False)
        ticker_data.reset_index(inplace=True)
        real, percent = get_diff(ticker_data)

        ticker_data["Real_Change"] = real
        ticker_data["Percent_Change"] = percent

        ticker_gain = ticker_data[ticker_data["Percent_Change"] > gain_cutoff]
        ticker_loss = ticker_data[ticker_data["Percent_Change"] < -loss_cutoff]

        pre_gain = get_pre_change_posts(ticker, ticker_gain, days, limit)
        pre_loss = get_pre_change_posts(ticker, ticker_loss, days, limit)

        gain_wc = dict(sorted(word_counts(pre_gain).items(), key=lambda x: x[1], reverse=True))
        loss_wc = dict(sorted(word_counts(pre_loss).items(), key=lambda x: x[1], reverse=True))

        gain_freq = remove_infrequent_words(gain_wc)
        loss_freq = remove_infrequent_words(loss_wc)

        gain_only, loss_only = remove_shared_keys(gain_freq, loss_freq)
    except Exception as e:
        print(e)
        return {}, {}
    
    return gain_only, loss_only

#gain_only, loss_only = process_ticker("FB")

#gain_only

#loss_only
"""
all_gain = []
all_loss = []
for ticker in ["TWTR", "FB", "MSFT", "ADBE", "AAPL", "SNAP", "AMZN", "NCL", "DIS", "NFLX"]:
    gain_only, loss_only = process_ticker(ticker)
    all_gain.append(gain_only)
    all_loss.append(loss_only)
    print(ticker)

def combine_dict_list(list_of_dicts):
    single_dict = {}
    for d in list_of_dicts:
        for i in d:
            if(i not in single_dict):
                single_dict[i] = d.get(i)
            else:
                single_dict.update({i:single_dict.get(i)+d.get(i)})
    return dict(sorted(single_dict.items(), key=lambda x: x[1], reverse=True))

gain, loss = remove_shared_keys(combine_dict_list(all_gain), combine_dict_list(all_loss))

gain

loss"""
_ = None # This is just to stop automatic output of block commented code

## Better Method (Probably)

Instead of looking at posts/comments the day before and predicting whether the next day will close higher, this will be looking at the posts/comments from the previous day's close to the current day's open and predicting whether the close price will be higher than the open price. 

In [8]:
def daily_change(data):
    change = []
    up = []
    for index, row in data.iterrows():
        change.append(row["Close"]-row["Open"])
        up.append(int(change[-1] > 0))
    return change, up

In [9]:
change, up = daily_change(twtr)

In [10]:
twtr["Daily_Change"] = change
twtr["Positive_Change"] = up

In [11]:
twtr

,Open,High,Low,Close,Adj Close,Volume,Daily_Change,Positive_Change
Date,,,,,,,,
2013-11-07,45.099998,50.090000,44.000000,44.900002,44.900002,117701600,-0.199997,0
2013-11-08,45.930000,46.939999,40.689999,41.650002,41.650002,27925300,-4.279999,0
2013-11-11,40.500000,43.000000,39.400002,42.900002,42.900002,16113900,2.400002,1
2013-11-12,43.660000,43.779999,41.830002,41.900002,41.900002,6316700,-1.759998,0
2013-11-13,41.029999,42.869999,40.759998,42.599998,42.599998,8688300,1.570000,1
...,...,...,...,...,...,...,...,...
2022-04-04,47.869999,51.369999,46.860001,49.970001,49.970001,268465400,2.100002,1
2022-04-05,53.849998,54.570000,50.560001,50.980000,50.980000,217520100,-2.869999,0
2022-04-06,50.040001,52.869999,49.299999,50.770000,50.770000,159034700,0.730000,1


In [14]:
twtr = twtr.sample(frac=1)

In [15]:
def get_pre_open_content(data, ticker, start_hour_diff=0, subreddit="stocks,stockmarket,stocksandtrading,daytrading,investing,stocks_picks,stockstobuytoday", limit=100):

    
    new_col = []
    for index, row in data.iterrows():
        end_time = row.name + timedelta(hours=9, minutes=30)
        start_time = end_time - timedelta(hours=17, minutes=30)
        content = []
        for i in api.search_comments(after=start_time, before=end_time, subreddit=subreddit, q=ticker, filter=['url','author', 'title', 'subreddit'], limit=limit):
            for j in i.body.split("."):
                for k in j.split("\n"):
                    content.append(k)
        new_col.append(content)
    
    return new_col

In [33]:
# This takes quite a while to run. Looking into how to speed it up. 
try:
    twtr = pd.read_csv("data\\twtr.csv")
except:
    twtr["Comments"] = get_pre_open_content(twtr, "TWTR", limit=100)

In [34]:
twtr

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Change,Positive_Change,Comments
0,2016-05-20,14.200000,14.510000,14.160000,14.430000,14.430000,18497400,0.230000,1,[]
1,2016-02-18,18.100000,18.549999,17.520000,18.430000,18.430000,34586300,0.330000,1,[]
2,2019-08-27,42.009998,42.689999,41.529999,42.139999,42.139999,16087500,0.130001,1,['[deleted]']
3,2015-04-13,51.990002,52.290001,51.450001,51.619999,51.619999,12306500,-0.370003,0,[]
4,2015-06-05,36.790001,37.230000,36.650002,37.000000,37.000000,11854300,0.209999,1,"[""I just randomly find them because I'm a mark..."
...,...,...,...,...,...,...,...,...,...,...
2115,2019-10-24,31.860001,32.389999,30.510000,30.750000,30.750000,105112500,-1.110001,0,[]
2116,2016-11-04,17.520000,18.340000,17.500000,18.020000,18.020000,30988700,0.500000,1,[]
2117,2016-09-28,23.420000,23.629999,22.440001,22.959999,22.959999,44513700,-0.460001,0,"['Remember, it can all be wiped out in a secon..."
2118,2020-08-26,40.549999,41.330002,40.000000,41.080002,41.080002,12532500,0.530003,1,"['ICLN 24%', '', 'ATVI 22%', '', 'KO 20%', '',..."


### Preparing text for analysis

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
def split_sequences(data):
    seqs = []
    vals = []
    for index, row in data.iterrows():
        for comment in row["Comment_Sequences"]:
            if(comment != []):
                seqs.append(list(comment))
                vals.append(row["Positive_Change"])
    return seqs, vals

In [20]:
def prepare_text(data, train_proportion = 0.8, max_len=20):
    tokenizer = Tokenizer(oov_token = "<OOV>")
    
    train = data[:int(data.shape[0]*train_proportion)]
    test = data[int(data.shape[0]*train_proportion):]
    
    for comment in train.Comments:
        tokenizer.fit_on_texts(comment)
        
    seqs = []
    for comment in train.Comments:
        seqs.append(tokenizer.texts_to_sequences(comment))
    
    train["Comment_Sequences"] = seqs
    
    X_train, y_train = split_sequences(train)
    
    X_train = pad_sequences(X_train, padding="post", truncating="post", maxlen=max_len)
    
    seqs = []
    for comment in test.Comments:
        seqs.append(tokenizer.texts_to_sequences(comment))
    
    test["Comment_Sequences"] = seqs
    
    X_test, y_test = split_sequences(test)
    
    X_test = pad_sequences(X_test, padding="post", truncating="post", maxlen=max_len)
    
    return np.array(X_train.tolist()), np.array(X_test.tolist()), np.array(y_train), np.array(y_test), tokenizer

In [21]:
max_len = 100

In [22]:
X_train, X_test, y_train, y_test, tokenizer = prepare_text(twtr, max_len=max_len)

C:\Users\zdude\AppData\Local\Temp\ipykernel_21880\2413103618.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Comment_Sequences"] = seqs
C:\Users\zdude\AppData\Local\Temp\ipykernel_21880\2413103618.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Comment_Sequences"] = seqs


### Creating TensorFlow Network

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 16, input_length=max_len))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dense(48, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           121440    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 48)                1200      
                                                                 
 dense_2 (Dense)             (None, 1)                 49        
                                                                 
Total params: 123,097
Trainable params: 123,097
Non-trainable params: 0
__________________________________________________

In [25]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), verbose=2)

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosi

Epoch 52/100
236/236 - 0s - loss: 0.2213 - accuracy: 0.8854 - val_loss: 2.1209 - val_accuracy: 0.4944 - 390ms/epoch - 2ms/step
Epoch 53/100
236/236 - 0s - loss: 0.2198 - accuracy: 0.8848 - val_loss: 2.1093 - val_accuracy: 0.4955 - 391ms/epoch - 2ms/step
Epoch 54/100
236/236 - 0s - loss: 0.2133 - accuracy: 0.8888 - val_loss: 2.0832 - val_accuracy: 0.5101 - 406ms/epoch - 2ms/step
Epoch 55/100
236/236 - 0s - loss: 0.2218 - accuracy: 0.8812 - val_loss: 2.1402 - val_accuracy: 0.4972 - 396ms/epoch - 2ms/step
Epoch 56/100
236/236 - 0s - loss: 0.2164 - accuracy: 0.8860 - val_loss: 2.1094 - val_accuracy: 0.5039 - 387ms/epoch - 2ms/step
Epoch 57/100
236/236 - 0s - loss: 0.2177 - accuracy: 0.8858 - val_loss: 2.1194 - val_accuracy: 0.5034 - 376ms/epoch - 2ms/step
Epoch 58/100
236/236 - 0s - loss: 0.2121 - accuracy: 0.8898 - val_loss: 2.1694 - val_accuracy: 0.4921 - 376ms/epoch - 2ms/step
Epoch 59/100
236/236 - 0s - loss: 0.2159 - accuracy: 0.8853 - val_loss: 2.1987 - val_accuracy: 0.4933 - 411ms/e

In [26]:
pred = model.predict(X_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [27]:
real_pred = []
for i in pred:
    real_pred.append(1 if i > 0.5 else 0)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(real_pred, y_test)

0.49551066217732886

### Evaluating Model

In [ ]:
def evaluate(data, model, train_proportion=0.8):
    days = []
    actual = []
    pred_proportion = []
    pred_val = []
    
    for date in data.Date.unique():
        days.append(date)
        actual.append()
        day = data[data["Date"] == date]
        predictions = []
        